# Librearias

In [17]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
import pickle

# Lectura de datos

In [2]:
df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
#Reconociendo los tipos de datos
df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [5]:
#Separando las columnas de las variables en categoricas y numericas
numericas= df.drop(['HeartDisease'], axis=1).select_dtypes('number').columns

categoricas = df.select_dtypes('object').columns

# Preprocesamiento

In [23]:
encoder = OneHotEncoder()
column_transformer = make_column_transformer((encoder,categoricas), remainder='passthrough')  

## Separacion del conjunto de entrenamiento y de test

In [24]:
x = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# Construccion y entrenamiento del modelo

In [25]:
model = LGBMClassifier(random_state=0)

## Pipeline

In [27]:
pipe = make_pipeline(column_transformer, model)

In [28]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object'))])),
                ('lgbmclassifier', LGBMClassifier(random_state=0))])

# Evaluacion del modelo

In [29]:
y_pred = pipe.predict(x_test)

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82       113
           1       0.87      0.90      0.88       163

    accuracy                           0.86       276
   macro avg       0.86      0.85      0.85       276
weighted avg       0.86      0.86      0.86       276



# Exportacion del modelo y encoder

In [19]:
pickle.dump( model, open( "model_file.pkl", "wb" ) )

In [20]:
pickle.dump( encoder, open( "encoder_file.pkl", "wb" ) )

In [21]:
pickle.dump( pipe, open( "pipe_file.pkl", "wb" ) )